<a id="top"></a>
# Exception Report Checklist - WFC3
***
## Learning Goals
In this notebook, you will walk through the procedure for addressing a HST/WFC3 exception report and <br> 
determining if a HOPR and/or Help Desk Ticket should be filed. 

By the end of this notebook, you will:

- Download your data.
- Display the data.
- Investigate data quality & check for anomalies.
- Inspect JIF (jitter/observing log) header keywords for problems.
- Determine if a HOPR is warranted.

## Table of Contents

[Introduction](#intro) <br>
[1. Imports](#import) <br>
[2. Download Data](#download) <br>
[3. View Data](#viewdata) <br>
[4. Header Keywords from `_jif.fits`](#keywords) <br>
[5. Plotting Data from `_jit.fits`](#jit) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[5.1 Interpreting the Jitter Plots](#interpret)<br>
[6. Further Analysis](#morework)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[6.1 Radial Profile Plots](#radprof)<br>
[7. Should I submit a HOPR or Help Desk Ticket?](#HOPR)<br>
[8. Conclusions](#conclusions) <br>
[Additional Resources](#add) <br>
[About this Notebook](#about) <br>
[Citations](#cite) <br>


## Introduction <a id="intro"></a>

This is a dedicated, stand-alone, notebook to help you look into your HST/WFC3 [Exception Report](https://www.stsci.edu/hst/observing/post-observation/exception-reports). <br>
HST observations go through a series of automated data quality checks. If a problem is found, an <br> 
Exception Report email will be sent to the Principal Investigator (PI). It is then the responsibility of the <br> 
PI to assess the data quality and determine whether or not the observations need to be repeated to <br> 
accomplish the science goals. <br>

Before starting this notebook, you should have received an Exception Report email from the STScI <br>
Automatic  AlertPI Exception Report Generator detailing which of your Exposure IDs were  affected. <br>
In this notebook, we provide all the necessary steps for directly downloading your data (including <br>
observing logs i.e. [JIF and JIT files](https://www.stsci.edu/hst/instrumentation/focus-and-pointing/pointing/jitter-file-format-definition)), displaying and analyzing it, and determining if a [Hubble <br> Observation Problem Report (HOPR)](https://www.stsci.edu/hst/observing/post-observation/reporting-problems) is warranted.
**There is a 90-day time limit from the date that <br> the data were delivered to file a HOPR.** Please keep in mind that when filing a HOPR, you will be <br>
required to show justification as to why the scientific goal(s) will be unattainable. If you run into any <br>
problems trying to use this notebook please contact the [WFC3 Help Desk](https://stsci.service-now.com/hst).<br>



## Examples of Nominal UVIS and IR Staring Mode Images  

<img align="left" width="1500" height="1200" src="docs/typical_images.png">

## Comparison of Nominal UVIS `_flt.fits` and `_flc.fits` Files <br> 
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `_flt.fits` are calibrated with no CTE correction  <br> 
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  `_flc.fits` are calibrated with a CTE correction (UVIS only)

<img align="left" width="600" height="600" src="docs/flt_vs_flc.gif">


**Note: The above animated GIF is generated from a 500x500 pixel subsection of the full-frame image. For** <br>
**more information about UVIS CTE please see** [Chapter 6](https://hst-docs.stsci.edu/wfc3dhb/chapter-6-wfc3-uvis-charge-transfer-efficiency-cte) **of the WFC3 Data Handbook and our** [WFC3 CTE <br>Performance Webpage](https://www.stsci.edu/hst/instrumentation/wfc3/performance/cte).

## 1. Imports <a id="import"></a>

Installation instructions for this notebook are in a `README.md` attached to the repository. <br>
Please make sure you have read the contents of the `README.md` before continuing the notebook
    
We import: 

- *glob* to make lists of files
- *os* to name files and remove directories 
- *astropy.io.fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *astroquery.mast.Observations* for downloading data from MAST
- *matplotlib.pyplot* for plotting data
- *display_image* for displaying any type of WFC3 image

In [ ]:
%matplotlib inline
import glob 
import os

from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations
import matplotlib.pyplot as plt

from docs.display_image import display_image


## 2. Download Data <a id="download"></a>

Here, we download the example images via `astroquery`. For more information, please look at the <br>
documentation for 
[Astroquery](https://astroquery.readthedocs.io/en/latest/),
[Astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html), and 
[CAOM Field Descriptions](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html), which is used for the <br>
`obs_table` variable. Additionally, you may download the data from MAST using either the [HST MAST<br>
Search Engine](https://mast.stsci.edu/search/hst/ui/#/) or the more general [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html).

We download `_flt.fits` images of star cluster NGC-1898 located in the Large Magellanic Cloud, <br>
from GO program [16748](https://www.stsci.edu/cgi-bin/get-proposal-info?id=16748&submit=Go&observatory=HST) as well as the corresponding `_jif.fits` and `_jit.fits` files. After <br>
downloading the images, we move them to a sub-directory within the current working directory. The <br>
data used in this notebook are courteous of PI Dr. Florian Niederhofer. <br>

If you would like to use this notebook to download your own data, edit line 1 in the cell below<br>
with your Exposure ID number from the exception report email. If your data are from WFC3/IR,<br>
you should also download the `_ima.fits` files by editing the variable `file_types` on line 4 <br>
to include `'IMA'` along with the `'FLT', 'JIF','JIT'` files. If your data are proprietary, <br>
you may need to take additional steps to download the data with astroquery.


In [ ]:
exp_ids = ['IEPP01010'] # Edit with exposure ID(s)

# Specify flle types to download
file_types = ['FLT', 'JIF','JIT']

#loop through exposure id
for obsid in exp_ids: 
    # make new directory to hold fits files - named by exposure id
    try:
        newdir = os.getcwd()+'/'+ obsid.lower()+'/'
        mkdir = os.mkdir(newdir)
        print(f'Making new directory {newdir}')
    except FileExistsError: 
        pass
    
    # loop through to get FLTs, JIFs, and JITs
    for file_type in file_types:
        print(f'Working on getting {file_type} files for Exposure ID {obsid}')
        obs_table = Observations.query_criteria(obs_id=obsid.lower())
        products = Observations.get_product_list(obs_table)
        filtered_products = Observations.filter_products(products, productSubGroupDescription=file_type, project='CALWF3')
        download_table = Observations.download_products(filtered_products, mrp_only=False)
    
        # For convenience move raws to cwd and remove empty download dir
        for file in download_table['Local Path']:
            filename = file.split('/')[-1]
            print(f'Moving {file} to {newdir+filename}')
            os.rename(file, newdir+filename)
            try:
                os.rmdir(f'mastDownload/HST/{filename[:9]}')
                print(f'Removing mastDownload/HST/{filename[:9]}')   
                os.rmdir(f'mastDownload/HST/{obsid.lower()}')
                print(f'Remvoing mastDownload/HST/{obsid.lower()}') 
            except (OSError, FileNotFoundError): 
                pass
            
    print(f'Remvoing mastDownload/HST/')    
    os.rmdir(f'mastDownload/HST/')
    print(f'Remvoing mastDownload/')
    os.rmdir('mastDownload/')
    

## 3. View Data <a id="viewdata"></a>

Below we use the imported [`display_image`](https://github.com/spacetelescope/WFC3Library/blob/master/notebooks/image_displayer_analyzer/display_image.py) function to display the data downloaded above. This <br> 
function will display the SCI, ERR, and DQ array for any full-frame or subarray WFC3 UVIS or IR image. <br> 
See our [notebook on displaying and analyzing data](https://github.com/spacetelescope/WFC3Library/tree/master/notebooks/image_displayer_analyzer) for different use cases and functionality. Additionally, <br>
the docstring for the function is available by running the command `display_image?`. The speed at <br>
which the images are displayed will depend partly on the number of files and their sizes.<br>

While viewing your data and assessing the quality, please keep in mind that the UVIS and IR detectors <br>
both have their own unique anomalies. For a full listing of all known WFC3 anomalies, as well as their <br>
description and example image, please see [WFC3 ISR 2017-22](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2017/WFC3-2017-22.pdf). Additionally, there is a [WFC3 Anomalies <br>webpage](https://www.stsci.edu/hst/instrumentation/wfc3/performance/anomalies), which also provides examples as well as linked documentation to various WFC3 Instrument <br>
Science Reports.

Please keep in mind that while the data may look OK to the eye, further analysis into any available PSFs <br>
as well as the S/N and/or flux achieved in the images may be required. For example, this notebook only <br>
shows images from the UVIS detector; if your data are from WFC3/IR please also download the <br>
Intermediate MultiAccum files (`_ima.fits`). These files contain the full stack of calibrated readouts, <br>
while the final calibrated image (`_flt.fits`) file is the single image fully processed and trimmed. <br>
For more information about the `_ima.fits` files see [Chapter 3.3](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps) of the WFC3 Data Handbook. To <br>
display each individual read of the `_ima.fits` file, turn the `ima_multiread` keyword to `True` in <br>
the `display_image` function. If you would like to remove one of the reads from the `_ima.fits` <br>
files and recalibrate with `calwf3` please see our [Jupyter Notebook](https://github.com/spacetelescope/WFC3Library/tree/master/notebooks/calwf3_recalibration) and [WFC3 ISR 2016-16](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2016/WFC3-2016-16.pdf) on manual <br>
recalibration.  

In [ ]:
# only run this cell if you would like to see the docscring for the function
display_image?

In [ ]:
# Edit fltfiles if you would like to use your own files
fltfiles = sorted(glob.glob(f'{exp_ids[0].lower()}/*flt.fits'))

for f in fltfiles:
    display_image(f,
                colormaps=['Greys_r', 'Greys_r', 'inferno_r'],
                scaling=[(-10, 130), (None, None), (None, None)],
                printmeta=True,
                ima_multiread=False,
                figsize=(16, 16),
                dpi=150)

<a id= "keywords"> </a> 
## 4. Header Keywords from `_jif.fits`
In the example above, we can see that the telescope guiding failed in at least 4 of the 7 images <br> 
from dataset IEPP01010. Since this is not a SNAP program  (which does not get repeat observations <br> per STScI policy) there is already enough evidence to file a [HOPR](https://www.stsci.edu/hst/observing/post-observation/reporting-problems) and request that the exposures <br> 
get repeated.

But, if you are still unsure if the data were affected, the next step is to look at the header keywords <br> 
The [jitter files](https://www.stsci.edu/hst/instrumentation/focus-and-pointing/pointing/jitter-file-format-definition) `_jif.fits`, also known as observing logs, record a lot of information including <br> 
Jitter, Pointing Control System data, World Coordinate System parameters, and spacecraft data <br> 
taken during the observations. <br> 

One of the sections in the header of the jitter file is called "Problem Flags and Warnings" where <br> 
a majority of the keywords will help assess if there were problems with guide star lock and tracking.<br>

If you would like to use your own `_jif.fits` file please edit line 1 in the cell below. The indexing<br>
on line 2 may also need to be modified.


In [ ]:
jif_file = f'{exp_ids[0].lower()}/{exp_ids[0].lower()}_jif.fits'  # Edit with the path to your own _jif.fits file
fits.getheader(jif_file,0)[-21:-6]

<br>Specifically from the keywords above, `T_GSFAIL` indicates one or more exposures had a guide <br>
star acquisition failure. 

In [ ]:
fits.getheader(jif_file,0)['T_GSFAIL*']

<br>Next, we loop through each extension of the `_jif.fits` file, and display various keywords that help <br> 
identify which exposures were affected. In the `_jif.fits` files, each extension number corresponds <br> 
to a given science exposure through the header keyword `EXPNAME`. In this example, the most relevant <br>
keywords that show something went wrong are `GUIDEACT`, `GSACQ`, `GSFAIL`, as well as `V2_RMS` <br>
and `V3_RMS`.

In [ ]:
numexts = fits.getheader(jif_file,0)['NEXTEND'] # number of extensions i.e. exposures

keywords = ['EXPNAME*','GUIDEACT*','GSACQ*','ACTGSSEP*',
            'GSSEPRMS*','NLOSSES*','CRVAL1*','CRVAL2*',
            'V2_RMS*','V3_RMS*','GSFAIL*']

for ext in range(1,numexts+1):
    print("JIF Header Ext Number:",ext)
    print('-'*80)
    for keyword in keywords:
        # try to display keyword because it may not be present 
        try: 
            print(fits.getheader(jif_file,ext)[keyword])
        except KeyError: 
            pass
    print('\n')

Above, we see that the `_jif.fits` file is reporting that exposures 4 through 7 failed to acquire a <br>
guide star. In extensions 4-7, the `GSACQ` keyword is blank, `GSFAIL` is set to `NOLOCK`, and the <br> 
values for `GSSEPRMS`, `V2_RMS`, `V3_RMS` are nonphysical at -32767 mas. <br> 

<a id= "jit"> </a> 
## 5. Plotting Data from `_jit.fits`

Next, we will use the data inside of the `_jit.fits` file to make plots of the jitter for each individual<br>
exposure. Specifically, we plot the V2 and V3 average jitter (over 3 seconds) as well as the V2 and V3 <br>
jitter RMS. The HST-based, or vehicle (V2, V3) system is an orthogonal reference frame tied to the <br>
telescope and is used operationally for alignment, pointing, and slewing purposes (V2 = -U2 and V3 <br>
= –U3). For more information about the coordinate systems, and UVIS and IR field geometries see the <br>
WFC3 Instrument Handbook [Chapter 6.4](https://hst-docs.stsci.edu/wfc3ihb/chapter-6-uvis-imaging-with-wfc3/6-4-uvis-field-geometry) and [Chapter 7.4](https://hst-docs.stsci.edu/wfc3ihb/chapter-7-ir-imaging-with-wfc3/7-4-ir-field-geometry) respectively. A description of each column,<br>
as well as the corresponding unit, can be found in the headers of the JIT file.<br>

If you would like to use your own `_jit.fits` file please edit line 1 in the cell below. Additionally, <br>
if you need to change the figure size and/or resolution please edit lines 4 and/or 5.

In [ ]:
jit_file = f'{exp_ids[0].lower()}/{exp_ids[0].lower()}_jit.fits' # Edit with the path to your own _jit.fits file
numexts = fits.getheader(jit_file,0)['NEXTEND'] # number of extensions i.e. exposures

figure_size = (7,5) # Edit if you want to chage the figure size
dotsperinch = 115   # Edit if you want to change the figure resolution

for ext in range(1,numexts+1):

    jit_tbl = Table(fits.getdata(jit_file,ext))
    expname = fits.getheader(jit_file,ext)['EXPNAME']
    flt_file = glob.glob(f"{exp_ids[0].lower()}/{expname[:8]}*flt.fits")
    
    plt.figure(figsize=figure_size, dpi=dotsperinch)
    plt.grid(alpha=0.5)
    plt.scatter(jit_tbl['Seconds'],jit_tbl['SI_V2_AVG'],15,alpha=.5,marker='o',label='V2_AVG')
    plt.scatter(jit_tbl['Seconds'],jit_tbl['SI_V3_AVG'],15,alpha=.5,marker='o',label='V3_AVG')
    plt.scatter(jit_tbl['Seconds'],jit_tbl['SI_V2_RMS'],10,alpha=.5,marker='s',label='V2_RMS')
    plt.scatter(jit_tbl['Seconds'],jit_tbl['SI_V3_RMS'],10,alpha=.5,marker='s',label='V3_RMS')
    
    
    plt.xlabel('Exposure Time [Seconds]',size=13)
    plt.ylabel('Coordinate Axis [Arcsec]',size=13)
    plt.title(f"Jitter File Ext Number: {ext}\n Corresponding FLT: {flt_file[0].split('/')[-1]}",size=14)
    plt.legend(prop={'size':12},ncol=2)
    plt.minorticks_on()

<a id="interpret"></a>
### 5.1 Interpreting the Jitter Plots 
In the plots above, there is more confirmation that exposures 4-7 failed to acquire a guide star. Every<br>
value in the V2 & V3 average and RMS columns inside of the `_jit.fits` file for those exposures <br>
are reporting an erroneous value of 1.6 x 10<sup>38</sup>. However, rather than having every value set to 1.6 x 10<sup>38</sup>, <br>
exposures 1-3 above have real jitter data reported. In this example, the average V2 & V3 jitter values <br>
vary by about +/- 0.5 - 2 mas and the RMS values are at a 1.5 - 2 mas level. 

At the time this notebook was written, the [HST Primer](https://hst-docs.stsci.edu/hsp/the-hubble-space-telescope-primer-for-cycle-30) section on [Optical Performance, Guiding <br> Performance, and Observing Efficiency](https://hst-docs.stsci.edu/hsp/the-hubble-space-telescope-primer-for-cycle-30/hst-primer-optical-performance-guiding-performance-and-observing-efficiency) states the dual guide star "pointing control method was <br>
designed to keep telescope jitter below 0.007 arcsec rms, but the current performance has jitter of <br>
0.008 arcsec rms." If the exposure had single guide star tracking, the Primer section explains that "a <br>
gyro drift around the guide star will be present that can be as large as 17 milliarcsec (mas) of roll <br> angle per second." 

<a id="morework"></a>
## 6. Further Analysis 
While we have determined that four of the exposures failed in this example, further analysis may be <br>
needed to access your data quality. <br>

If you are still unsure whether or not your data quality was affected (such as in exposures 1-3 above),<br>
we kindly ask that you complete a couple more verification steps:
1. Determine whether the signal-to-noise and/or flux of your data reached the expected levels
2. Analyze any point spread functions or sources that appear in your exposures for signs of drifting
<br> 

For analyzing the S/N and/or flux in your image using Python, we recommend the Astropy package for <br>
photometry, [photutils](https://photutils.readthedocs.io/en/stable/), and [imexam](https://imexam.readthedocs.io/en/0.9.1/#). Some of the methods offered by photutils are [source detection](https://photutils.readthedocs.io/en/stable/detection.html), <br>
[aperture photometry](https://photutils.readthedocs.io/en/stable/aperture.html), and [PSF photometry](https://photutils.readthedocs.io/en/stable/psf.html). The imexam method offers many useful diagnostic plots <br>
such as [radial profile](https://imexam.readthedocs.io/en/0.9.1/imexam/imexam_command.html#radial-profile-plot), [curve of growth](https://imexam.readthedocs.io/en/0.9.1/imexam/imexam_command.html#curve-of-growth-plot), and [contours](https://imexam.readthedocs.io/en/0.9.1/imexam/imexam_command.html#contour-plots). For more information about WFC3 PSF analysis <br>
please see our [PSF data analysis webpage](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/psf).


<a id="radprof"></a>
### 6.1 Radial Profile Plots

Below, we show an example of searching for sources with `photutils.detection.DAOStarFinder` <br>
in a 100x100 pixel subsection and subsequently plotting their radial profiles using the `RadialProfile` <br> class within the file `rad_prof.py`. 

We also import:
- *numpy* for handling arrays
- *astropy.stats.sigma_clipped_stats* for sigma clipping statistics
- *ginga.util.zscale* for scaling images
- *matplotlib.colors.LogNorm* for logarithmic normalization
- *photutils.detection.CircularAperture* for aperture photometry

In [ ]:
import numpy as np

from astropy.stats import sigma_clipped_stats
from ginga.util import zscale
from matplotlib.colors import LogNorm
from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder

from docs.rad_prof import RadialProfile

<br>

First, we open the data and 3 sigma clip it to get the median and standard deviation values. We then use <br>
those values to help search for sources using `photutils.detection.DAOStarFinder`. We truncate <br>
the list of sources to those with `flux > 10` and well within the x & y border of the 100x100 pixel <br>
subsection. Lastly, we create `photutils.aperture` objects so we can easily display the subsection <br>
with each source marked. 

In [ ]:
# Read in data
filename = 'iepp01010/iepp01uvq_flt.fits'
uvis2 = fits.getdata(filename,'SCI',1)
header = fits.getheader(filename)

# Trim data to 100x100 subsection
data = uvis2[:100,65:165]

# 3 sigma clip data to get median and std values
mean, median, std = sigma_clipped_stats(data, sigma=3.0) 

# Search for sources
daofind = DAOStarFinder(fwhm=2.5, threshold=3*std)  
sources = daofind(data - median)  

# Truncate list to show just a few sources  
sources = sources[(sources['flux'] > 10) &\
                  (sources['xcentroid'] > 10) & (sources['xcentroid'] < 90) &\
                  (sources['ycentroid'] > 18) & (sources['ycentroid'] < 90) ]

# Create circular apertures to plot
positions = np.transpose((sources['xcentroid'],sources['ycentroid']))
apertures = CircularAperture(positions, r=5.)

# Get zscale image min and max limits
z1,z2 = zscale.zscale(data)

# Plot 100x100 subsection and apertures
plt.figure(figsize=(15,10))
im1 = plt.imshow(data-z1+.01, origin='lower', cmap='Greys', norm = LogNorm(vmin=.01, vmax=z2*100.-z1) )
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.title(filename,size=14)
plt.colorbar(im1,pad=0.01)


<br>

Now that we have the x & y coordinates for several (12) sources in the image, we loop through each one and<br>
calculate and display the radial profile, which provides the measured FWHM value in the legend of the plot. <br>
**For more information about UVIS and IR optical performance, such as nominal PSF FWHM values, please <br> 
see WFC3 Instrument Handbook Chapters [6.6](https://hst-docs.stsci.edu/wfc3ihb/chapter-6-uvis-imaging-with-wfc3/6-6-uvis-optical-performance) and [7.6](https://hst-docs.stsci.edu/wfc3ihb/chapter-7-ir-imaging-with-wfc3/7-6-ir-optical-performance) respectively.**

In [ ]:
# Loop through sources and plot star stamp next to corresponding radial profile plot
for xy in zip(sources['xcentroid'],sources['ycentroid']):
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize=(11,5))
    
    # Calculate radial profile and plot on ax2
    my_prof = RadialProfile(xy[0],xy[1],data,
                  r=5,
                  fit=True,
                  recenter=True,
                  show=True,
                  ax=ax2)
    
    # Create boundaries for stamp 
    x1 = int(round(my_prof.x-7))
    x2 = int(round(my_prof.x+7))
    y1 = int(round(my_prof.y-7))
    y2 = int(round(my_prof.y+7))
    
    # Plot star stamp 
    im1 = ax1.imshow(data[y1:y2,x1:x2]-z1+.01, origin='lower', cmap='Greys', extent= [x1,x2,y1,y2],norm = LogNorm(vmin=.01, vmax=z2*100.-z1) )

    ax1.set_title(f"x = {my_prof.x:.3f}, y = {my_prof.y:.3f}",size=13)
    ax2.set_title(header['filter'],size=13)
    ax2.grid(alpha=0.5)
    fig.tight_layout()

**Radial Profile Plot Note:** The scatter points that trend above the fitted line between pixel distances ~2-4, and<br>
then below the fitted line between ~4-7 pixels are due to the Airy rings surrounding the PSF. 

<a id="HOPR"></a>
## 7. Should I submit a [HOPR](https://www.stsci.edu/hst/observing/post-observation/reporting-problems) or [Help Desk Ticket](https://stsci.service-now.com/hst)? 

If your data from the exception report are affected by a problem, as in the example above, please file a [HOPR](https://www.stsci.edu/hst/observing/post-observation/reporting-problems)<br>
and begin the process of requesting a repeat observation. If you are still unsure whether or not your data are <br>
affected by an issue after displaying it, checking the header keywords in the observing logs, plotting the jitter<br>
data, a quick-look determination of the S/N, and checking any PSFs for signs of drift, please reach out to the  <br>
[WFC3 Help Desk](https://stsci.service-now.com/hst). Lastly, please do not forget that **there is a 90-day time limit from the date that the data <br>
were delivered to file a HOPR.**

<a id="conclusions"></a>
## 8. Conclusions 

Thank you for going through this notebook. You should now have all the necessary tools for accessing the <br>
data from your exception report. After completing this notebook you should be more familiar with: <br>

- Downloading your data.
- Displaying your data.
- Viewing the `_jif.fits` file header.
- Plotting data from the `_jit.fits` file.
- Determining if a HOPR should be filed.

**Congratulations, you have completed the notebook.**

## Additional Resources <a id="add"></a>

Below are some additional resources that may be helpful. Please feel free to contact the [WFC3 Helpdesk](https://stsci.service-now.com/hst)<br>
with any questions.

- [WFC3 Website](https://www.stsci.edu/hst/instrumentation/wfc3)
- [WFC3 Instrument Handbook](https://hst-docs.stsci.edu/wfc3ihb)
- [WFC3 Data Handbook](https://hst-docs.stsci.edu/wfc3dhb)
- [WFC3 Anomalies](https://www.stsci.edu/hst/instrumentation/wfc3/performance/anomalies)
- [WFC3 PSF Data Analysis](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/psf)
- [Jitter File and Format](https://www.stsci.edu/hst/instrumentation/focus-and-pointing/pointing/jitter-file-format-definition)

## About this Notebook <a id="about"></a>

**Author:** Benjamin Kuhn, WFC3 Instrument

**Updated On:** January 20, 2023

## Citations <a id="cite"></a>

If you use Python packages for published research, please cite the authors. Follow these links for more <br>
information about citing packages such as `astropy`, `astroquery`, `matplotlib`, or `photutils`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/citing.html)
* [Citing `photutils`](https://photutils.readthedocs.io/en/stable/citation.html)
<br>
***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 